In [ ]:
import requests
import json
import csv
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs

TGOS_URL = 'https://api.tgos.tw/TGOS_MAP_API/Docs/Example/99'

def get_keystr():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get(TGOS_URL)
        driver.refresh()
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it("iframeResult"))
        search_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//input[@type="submit" and @value="地址定位"]'))
        )
        search_button.click()
        time.sleep(5)
        network_requests = driver.execute_script("""
            var performance = window.performance || {};
            return performance.getEntries() || [];
        """)
        for request in network_requests:
            url_str = request.get('name', '')
            if 'TGA' in url_str and 'keystr=' in url_str:
                query_params = parse_qs(urlparse(url_str).query)
                keystr_list = query_params.get('keystr', [])
                if keystr_list:
                    return keystr_list[0]
    finally:
        driver.quit()
    return None

def query_address(address, keystr, session):
    try:
        params = {
            'oAddress': address,
            'oSRS': 'EPSG:3826',
            'oResultDataType': 'jsonp',
            'pnum': 'NaN',
            'keystr': keystr,
            'jsonp': "TGOS.getJSON['sn2']"
        }
        response = session.post("https://gis.tgos.tw/TGAddress/TGAddress.aspx", params=params)
        result = response.text.replace("TGOS.getJSON['sn2'](", "").rstrip(");")
        data = json.loads(result)
        if 'AddressList' in data and data['AddressList']:
            info = data['AddressList'][0]
            return [address, info.get('X', '-9999999'), info.get('Y', '-9999999')]
    except:
        pass
    return [address, '-9999999', '-9999999']

def main():
    ################################## file path ##########################
    input_path = r'C:\Users\USER\Desktop\test.csv'
    output_path = r'C:\Users\USER\Desktop\Addresss_result.csv'
    ########################################################################
    df = pd.read_csv(input_path, encoding="utf-8")
    ################## Name of the address column in the CSV file ##################
    address_list = df['商業地址'].dropna().tolist()
    #################################################################################
    keystr = get_keystr()
    last_keystr_time = time.monotonic()
    print(f"[INFO] Initial keystr obtained: {keystr}")

    headers = {
        'user-agent': 'Mozilla/5.0',
        'referer': 'https://map.tgos.tw/',
    }
    cookies = {
        '_ga': 'GA1.2.2139804078.1676556517',
        '_gid': 'GA1.2.651109627.1679290300',
    }

    results = []

    with requests.Session() as session:
        session.headers.update(headers)
        session.cookies.update(cookies)

        def safe_query(addr):
            nonlocal keystr, last_keystr_time
            if time.monotonic() - last_keystr_time > 1000:
                print("[INFO] Refreshing keystr...")
                new_keystr = get_keystr()
                if new_keystr:
                    keystr = new_keystr
                    last_keystr_time = time.monotonic()
                    print(f"[INFO] New keystr obtained: {keystr}")
            return query_address(addr, keystr, session)

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {
                executor.submit(safe_query, addr): addr
                for addr in address_list
            }
            for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
                try:
                    results.append(future.result())
                except Exception as e:
                    print(f"[ERROR] {e}")

    with open(output_path, 'w', newline='', encoding="utf_8_sig") as f:
        writer = csv.writer(f)
        writer.writerow(['Address', 'X', 'Y'])
        writer.writerows(results)

    print("✅ Done!")

if __name__ == "__main__":
    main()
